In [92]:
import pyodbc
import pandas as pd

server = '' 
database = '' 
username = '' 
password = '' 
tianHangConn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
#cursor = cnxn.cursor()

tianHang = pd.read_sql_query(
  '''SELECT * FROM dbo.sentimentCount''', tianHangConn)

In [93]:
server = '' 
database = '' 
username = '' 
password = '' 
sylvesterConn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

sylvester = pd.read_sql_query(
  '''SELECT NC.NewCases, NC.NewCasesSmoothed, NC.Date, CTY.CountryName FROM NewCases NC
    INNER JOIN Country CTY
    ON NC.CountryID = CTY.ID
    ''', sylvesterConn)

In [94]:
import datetime

def convertToTimestamp(string):
    return pd.Timestamp(datetime.datetime.strptime(string, "%Y-%m-%d"))

In [95]:
sylvester.iloc[0]['Date']

Timestamp('2020-03-13 00:00:00')

In [96]:
tianHang.iloc[0]['_c0']

'2020-03-09'

In [97]:
import time
import datetime
# s = "13/03/2020"
s = "2020-03-13"

# print(final == sylvester.iloc[0]['Date'])
# type(final)
# final

In [98]:
import numpy as np

def getSentimentValue(positive, negative):
    return (positive/ (positive + negative))

In [99]:
tianHang['Date'] = tianHang['_c0'].apply(convertToTimestamp)
tianHang['Sentiment'] = np.vectorize(getSentimentValue)(tianHang['positive_Count'], tianHang['negative_Count'])
tianHang

,_c0,_c2,positive_Count,negative_Count,netural_Count,Date,Sentiment
0,2020-03-09,Singapore,17,57,86,2020-03-09,0.229730
1,2020-03-09,Malaysia,23,74,125,2020-03-09,0.237113
2,2020-03-09,France,45,167,211,2020-03-09,0.212264
3,2020-03-09,Germany,31,114,96,2020-03-09,0.213793
4,2020-03-09,Japan,25,71,73,2020-03-09,0.260417
5,2020-03-09,Sweden,14,36,25,2020-03-09,0.280000
6,2020-03-09,Thailand,14,45,67,2020-03-09,0.237288
7,2020-03-09,China,8,38,43,2020-03-09,0.173913
8,2020-03-09,Philippines,69,201,338,2020-03-09,0.255556
9,2020-03-09,United States,343,1606,1199,2020-03-09,0.175988


In [100]:
tianHang['_c2'].unique()

array(['Singapore', 'Malaysia', 'France', 'Germany', 'Japan', 'Sweden',
       'Thailand', 'China', 'Philippines', 'United States',
       'United Kingdom', 'Canada', 'India', 'Hong Kong'], dtype=object)

# Cases vs sentiments

In [101]:
#filter by country
Country = 'Singapore'

filtered_Sylvester = sylvester[sylvester['CountryName'] == Country].copy()
filtered_tianHang = tianHang[tianHang['_c2'] == Country].copy()

plot = pd.merge(left=filtered_Sylvester, right=filtered_tianHang, left_on='Date', right_on='Date')
plot = plot[['Date','NewCases', 'Sentiment']]
plot = plot.reset_index()

# plot

In [102]:
import plotly.graph_objects as go

from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(go.Scatter(x=plot['Date'], y=plot['NewCases'],
                    mode='lines+markers',
                    name='New Cases'))

fig.add_trace(go.Scatter(x=plot['Date'], y=plot['Sentiment'],
                    mode='lines+markers',
                    name='Sentiment'),
                    secondary_y=True,)

# Add figure title
fig.update_layout(
    title_text="Cases"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Number of Cases per Day", secondary_y=False)
fig.update_yaxes(title_text="Sentiment Score", secondary_y=True)

fig.show()

In [103]:
casesQuery = '''SELECT NC.NewCases, NC.NewCasesSmoothed, NC.Date, CTY.CountryName FROM NewCases NC
    INNER JOIN Country CTY
    ON NC.CountryID = CTY.ID
    '''

deathsQuery = '''SELECT ND.NewDeaths, ND.NewDeathsSmoothed, ND.Date, CTY.CountryName FROM NewDeaths ND
INNER JOIN Country CTY
ON ND.CountryID = CTY.ID'''

testsQuery = '''SELECT NT.NewTests, NT.NewTestsSmoothed, NT.Date, CTY.CountryName FROM NewTests NT
INNER JOIN Country CTY
ON NT.CountryID = CTY.ID'''

bedsQuery = '''SELECT BPT.BedsPerThousand, BPT.Date, CTY.CountryName FROM BedsPerThousand BPT
INNER JOIN Country CTY
ON BPT.CountryID = CTY.ID'''

handwashQuery = '''SELECT HF.HandwashingShare, HF.Date, CTY.CountryName FROM HandwashingFacilities HF
INNER JOIN Country CTY
ON HF.CountryID = CTY.ID'''

stringencyQuery = '''SELECT ST.Stringency, ST.Date, CTY.CountryName FROM Stringency ST
INNER JOIN Country CTY
ON ST.CountryID = CTY.ID
'''

types = ['New Daily Covid-19 Cases', 'New Daily Covid-19 Deaths', 'New Daily Covid-19 Tests', 'Daily Hospital Beds per Thousand', 'Daily Basic Handwashing Facilities', 'Daily Government Response Stringency Index']
queries = [casesQuery, deathsQuery, testsQuery, bedsQuery, handwashQuery, stringencyQuery]
selectedQuery = ['NewCases', 'NewDeaths', 'NewTests', 'BedsPerThousand', 'HandwashingShare', 'Stringency']


In [104]:
import scipy.stats

#Pearson's r
pearson = scipy.stats.pearsonr(plot['NewCases'], plot['Sentiment'])

# Spearman's rho
spearman = scipy.stats.spearmanr(plot['NewCases'], plot['Sentiment'])

# Kendall's tau
kendall = scipy.stats.kendalltau(plot['NewCases'], plot['Sentiment'])

print(pearson, spearman, kendall)

(0.30823434840345704, 0.009975461792258158) SpearmanrResult(correlation=0.5148757653926053, pvalue=6.001734918475889e-06) KendalltauResult(correlation=0.3420644809202785, pvalue=4.6315313470641994e-05)


# Steps
1. Get ### of each country
2. Store this into separate DFs
3. Compare this to the sentiment
4. Store this into 1 DF for all scores per ###
5. Show countries that acted late


In [105]:
def loadSentimentData():
  tianHang = pd.read_sql_query(
  '''SELECT * FROM dbo.sentimentCount''', tianHangConn)
  tianHang['Date'] = tianHang['_c0'].apply(convertToTimestamp)
  tianHang['Sentiment'] = np.vectorize(getSentimentValue)(tianHang['positive_Count'], tianHang['negative_Count'])
  return tianHang

def loadCountryAnalysisData(query):
  sylvester = pd.read_sql_query(
  query, sylvesterConn)
  return sylvester

sentimentData = loadSentimentData()
country_list = list(sentimentData['_c2'].unique())
country_list

['Singapore',
 'Malaysia',
 'France',
 'Germany',
 'Japan',
 'Sweden',
 'Thailand',
 'China',
 'Philippines',
 'United States',
 'United Kingdom',
 'Canada',
 'India',
 'Hong Kong']

In [142]:
df_template= pd.DataFrame(columns=['Country', 'Corr', 'Pearsons', 'Pearsons P value', 'Spearman', 'Spearman P value', 'Kendall', 'Kendall P value'])

df_cases = df_template.copy()
df_deaths = df_template.copy()
df_tests = df_template.copy()
df_beds = df_template.copy()
df_handwash = df_template.copy()
df_stringency = df_template.copy()

df_list = [df_cases, df_deaths, df_tests, df_beds, df_handwash, df_stringency]

country_list = ['Singapore',
 'Malaysia',
 'France',
 'Germany',
 'Japan',
 'Sweden',
 'Thailand',
 'China',
 'Philippines',
 'United States',
 'United Kingdom',
 'Canada',
 'India',
 'Hong Kong']

for query in queries:
    for Country in country_list:
        countryAnalysisData = loadCountryAnalysisData(query)
        filteredCountryAnalysisData = countryAnalysisData[countryAnalysisData['CountryName'] == Country].copy()
        filteredSentimentData = sentimentData[sentimentData['_c2'] == Country].copy()
        calculate = pd.merge(left=filteredCountryAnalysisData, right=filteredSentimentData, left_on='Date', right_on='Date')
        calculate = calculate[['Date',selectedQuery[queries.index(query)], 'Sentiment']]
        r = np.corrcoef(calculate[selectedQuery[queries.index(query)]], calculate['Sentiment'])
        correlationCoef = r[0, 1]
        pearsons = scipy.stats.pearsonr(calculate[selectedQuery[queries.index(query)]], calculate['Sentiment'])
        pearsonsPval = pearsons[1]
        pearsons = pearsons[0]
        spearman = scipy.stats.spearmanr(calculate[selectedQuery[queries.index(query)]], calculate['Sentiment'])
        spearmanPval = spearman[1]
        spearman = spearman[0]
        kendall = scipy.stats.kendalltau(calculate[selectedQuery[queries.index(query)]], calculate['Sentiment'])
        kendallPval = kendall[1]
        kendall = kendall[0]
        df_temp = pd.DataFrame([[Country, correlationCoef, pearsons, pearsonsPval , spearman, spearmanPval, kendall, kendallPval]], columns=['Country', 'Corr', 'Pearsons', 'Pearsons P value', 'Spearman', 'Spearman P value', 'Kendall', 'Kendall P value'])
        df_list[queries.index(query)] = df_list[queries.index(query)].append(df_temp)


In [143]:
index = 5

print(selectedQuery[index])
df_list[index].describe()

Stringency


,Corr,Pearsons,Pearsons P value,Spearman,Spearman P value,Kendall,Kendall P value
count,14.000000,14.000000,1.400000e+01,14.000000,1.300000e+01,14.000000,1.400000e+01
mean,0.619291,0.619291,3.511850e-03,0.681723,7.577993e-02,0.545226,1.398451e-01
std,0.305255,0.305255,1.020757e-02,0.240948,2.732262e-01,0.225052,3.555608e-01
min,-0.251335,-0.251335,0.000000e+00,-0.002285,5.012369e-19,-0.004696,6.688992e-14
25%,0.534399,0.534399,1.774778e-15,0.570446,3.303118e-18,0.453337,2.002108e-12
50%,0.685819,0.685819,1.239447e-10,0.751853,4.256874e-13,0.576731,4.967882e-08
75%,0.786355,0.786355,1.156674e-05,0.827768,1.114690e-06,0.662683,2.339648e-05
max,1.000000,1.000000,3.723586e-02,1.000000,9.851318e-01,1.000000,1.000000e+00


In [108]:
df_template= pd.DataFrame(columns=['Type', 'Corr', 'Pearsons', 'Pearsons P value', 'Spearman', 'Spearman P value', 'Kendall', 'Kendall P value'])

df_cty_list = []
for i in country_list:
    df_cty_list.append(df_template)


for Country in country_list:
    for query in queries:
        countryAnalysisData = loadCountryAnalysisData(query)
        filteredCountryAnalysisData = countryAnalysisData[countryAnalysisData['CountryName'] == Country].copy()
        filteredSentimentData = sentimentData[sentimentData['_c2'] == Country].copy()
        calculate = pd.merge(left=filteredCountryAnalysisData, right=filteredSentimentData, left_on='Date', right_on='Date')
        calculate = calculate[['Date',selectedQuery[queries.index(query)], 'Sentiment']]
        r = np.corrcoef(calculate[selectedQuery[queries.index(query)]], calculate['Sentiment'])
        correlationCoef = r[0, 1]
        pearsons = scipy.stats.pearsonr(calculate[selectedQuery[queries.index(query)]], calculate['Sentiment'])
        pearsonsPval = pearsons[1]
        pearsons = pearsons[0]
        spearman = scipy.stats.spearmanr(calculate[selectedQuery[queries.index(query)]], calculate['Sentiment'])
        spearmanPval = spearman[1]
        spearman = spearman[0]
        kendall = scipy.stats.kendalltau(calculate[selectedQuery[queries.index(query)]], calculate['Sentiment'])
        kendallPval = kendall[1]
        kendall = kendall[0]
        df_temp = pd.DataFrame([[query, correlationCoef, pearsons, pearsonsPval , spearman, spearmanPval, kendall, kendallPval]], columns=['Type', 'Corr', 'Pearsons', 'Pearsons P value', 'Spearman', 'Spearman P value', 'Kendall', 'Kendall P value'])
        df_cty_list[country_list.index(Country)] = df_cty_list[country_list.index(Country)].append(df_temp)

In [155]:

print(country_list)
print('\n\n\n')


index = 10

print(country_list[index])
df_cty_list[index]

['Singapore', 'Malaysia', 'France', 'Germany', 'Japan', 'Sweden', 'Thailand', 'China', 'Philippines', 'United States', 'United Kingdom', 'Canada', 'India', 'Hong Kong']




United Kingdom


,Type,Corr,Pearsons,Pearsons P value,Spearman,Spearman P value,Kendall,Kendall P value
0,"SELECT NC.NewCases, NC.NewCasesSmoothed, NC.Da...",6.775623e-01,6.775623e-01,1.617593e-10,0.894422,4.162524e-25,0.715461,2.623775e-17
0,"SELECT ND.NewDeaths, ND.NewDeathsSmoothed, ND....",5.214153e-01,5.214153e-01,4.350242e-06,0.828781,1.487303e-18,0.630766,1.765699e-12
0,"SELECT NT.NewTests, NT.NewTestsSmoothed, NT.Da...",4.145686e-01,4.145686e-01,3.977966e-04,0.420646,3.196719e-04,0.344930,3.543381e-04
0,"SELECT BPT.BedsPerThousand, BPT.Date, CTY.Coun...",7.518929e-17,2.443652e-16,1.000000e+00,NaN,NaN,NaN,NaN
0,"SELECT HF.HandwashingShare, HF.Date, CTY.Count...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"SELECT ST.Stringency, ST.Date, CTY.CountryName...",7.898642e-01,7.898642e-01,7.231661e-16,0.834757,5.012369e-19,0.663735,1.336386e-12
